In [5]:
import cv2
import numpy as np


# 面向对象 : 把grabcut进行交互式抠图的功能封装成一个类.


In [16]:
class App:
    def __init__(self, image):
        self.image = image
        self.img = cv2.imread(self.image)
        self.img2 = self.img.copy()
        self.start_x = 0
        self.start_y = 0
        # 是否需要绘制矩形的标志
        self.rect_flag = False
        self.rect = (0, 0, 0, 0)
        self.mask = np.zeros(shape=self.img.shape[:2], dtype=np.uint8)
        # 输出
        self.output = np.zeros(shape=self.img.shape[:2], dtype=np.uint8)
    
    # 实例方法, 第一个参数一定是self
    # staticmethod默认类和实例对象不会自动传参数(self, cls)
    # @staticmethod 静态方法   @classmethod
    def on_mouse(self, event, x, y, flags, param):
        # 按下左键, 开始框选前景区域
        if event == cv2.EVENT_LBUTTONDOWN:
            # 记录起始的坐标
            self.start_x = x
            self.start_y = y
            self.rect_flag = True
        elif event == cv2.EVENT_LBUTTONUP:
            self.rect_flag = False
            # 记录用户的矩形大小
            self.rect = (min(self.start_x, x), min(self.start_y, y),
                         abs(self.start_x - x), abs(self.start_y - y))
            cv2.rectangle(self.img, (self.start_x, self.start_y), (x, y), (0, 0, 255), 2)
        elif event == cv2.EVENT_MOUSEMOVE and self.rect_flag:
            # 画矩形
            self.img = self.img2.copy()
            cv2.rectangle(self.img, (self.start_x, self.start_y), (x, y), (0, 255, 0), 2)
            
    # 编辑模式
    # 核心逻辑: 窗口 回调函数 图片
    def run(self):
        cv2.namedWindow('img')
        # 绑定鼠标事件
        cv2.setMouseCallback('img', self.on_mouse)
        while True:
            cv2.imshow('img', self.img)
            cv2.imshow('output', self.output)
            
            key = cv2.waitKey(1)
            if key == 27:
                break
            elif key == ord('g'):
                # 进行切图
                cv2.grabCut(self.img2, self.mask, self.rect, None, None, 5, 
                            mode=cv2.GC_INIT_WITH_RECT)
            # 把前景或者可能是前景的位置设置为255, 
            mask2 = np.where((self.mask == 1) | (self.mask == 3), 255, 0).astype(np.uint8)
            # 使用与运算.
            self.output = cv2.bitwise_and(self.img2, self.img2, mask=mask2)
        cv2.destroyAllWindows()
        
        
app = App('./lena.png')
app.run()